In [2]:
from PyPDF2 import PdfReader
import os
import pandas as pd
import glob

In [3]:
reader = PdfReader('test_data/not_relevant/PDF_art20329.pdf')
page = reader.pages[1]
text = page.extract_text()
len(reader.pages)

5

In [38]:
def get_pdfs_in_directory(directory_path):
    # Use glob to get all PDF files in the directory
    pdf_files = glob.glob(os.path.join(directory_path, '*.pdf'))
    return pdf_files
pdf_not_relevant = get_pdfs_in_directory('test_data/not_relevant')
pdf_relevant = get_pdfs_in_directory('test_data/relevant')

In [5]:
harvest_nlp = pd.DataFrame(all_info,columns=['num_pages','text','is_relevant'])
harvest_nlp

,num_pages,text,is_relevant
0,37,TECHNICAL REPORT\nEffectiveness and cost- \n...,0
1,45,SPECIAL REPORT\nImplementing the \nDublin Dec...,0
2,15,SPECIAL REPORT\nThematic report: \nSex worker...,0
3,36,TECHNICAL REPORT\nRisk assessment on HIV in G...,0
4,9,\nSuggested citation: European Centre for Dis...,0
...,...,...,...
67,11,Suggested citation: HIV testing in Europe and ...,1
68,8,\n \n \nSuggested citation: European Centre f...,1
69,13,\n \nEuropean Centre for Disease Prevention a...,1
70,69,SPECIAL REPORT\nContinuum of HIV care\n \nMoni...,1


In [43]:
def text_extract(reader):
    doc_text = ""
    num_pages = len(reader.pages)
    for i in range(num_pages):
        page_text = reader.pages[i].extract_text() + " "
        doc_text += page_text
    return doc_text

def get_info(path,relevant):
    with open(path, 'rb') as f:
        pdf = PdfReader(f)
        info = pdf.metadata
        print(info)
        author = info.author if '/Author' in info else None
        title = info.title if '/Title' in info else None
        subject = info.subject if '/Subject' in info else None
        keywords = info['/Keywords'] if '/Keywords' in info else None
        number_of_pages = len(pdf.pages)
        text = text_extract(pdf)
        relevance = 1 if relevant else 0
    return [author, title, subject, keywords, number_of_pages, text, relevance]

In [44]:
get_info(pdf_not_relevant[0],True)

{'/Author': 'ECDC', '/CreationDate': "D:20170310153714+01'00'", '/Creator': 'Microsoft® Word 2013', '/ModDate': "D:20170310163808+01'00'", '/Producer': 'Microsoft® Word 2013', '/Title': 'Effectiveness and cost-effectiveness of antenatal screening for HIV, hepatitis B, syphilis and rubella susceptibility'}


['ECDC',
 'Effectiveness and cost-effectiveness of antenatal screening for HIV, hepatitis B, syphilis and rubella susceptibility',
 '',
 '',
 37,
 'TECHNICAL  REPORT\nEffectiveness and cost-  \neffectiveness of antenatal screening \nfor HIV, hepatitis B, syphilis and  \nrubella susceptibility\n \nLiterature review\nwww.ecdc.europa.eu ECDC  TECHNICAL REPORT  \nEffectiveness and cost -effectiveness of \nantenatal screening for HIV, hepatitis B, \nsyphilis and rubella susceptibility  \nLiterature  review  \n \n \n \n \n \n \n \n \n \n  \n ii This report was commissioned by the European Centre for Disease Prevention and Control (ECDC) and coordinated \nby Otilia Mårdh, Tarik Derrough and Andrew Amato -Gauci.  \nThe report was produced under contract ECDC/2012/052 with the National Institute for Health and Welfare (T HL) \nby Carita Savolainen -Kopra, Mia Kontio, Marjukka Mäkelä, Kirsi Liitsola, Jukka Lindeman, Jaana Isojärvi, Heljä -Marja \nSurcel, Irja Davidkin, Henrikki Brummer -Korvenko

In [30]:
all_info = []
for pdf in pdf_not_relevant:
	all_info.append(get_info(pdf,relevant = False))
for pdf in pdf_relavent:
	all_info.append(get_info(pdf,relevant = True))
harvest_nlp = pd.DataFrame(all_info,columns = ['author','title','subject','keywords','num_pages','text','is_relavent'])

KeyError: '/Keywords'

In [ ]:
patterns = {
	### UNDERSTAND CONTEXT FIRST BEFORE CHANGING, HELPS TO MAKE MORE INFORMED DECISIONS
	#remove \n and \r and \t
	r'\n|\r|\t': '',

	#replace phone numbers to "convertedphonenumber" (phone number string may have different meaning)
	r'\d{3}-\d{3}-\d{4}': 'convertedphonenumber',

	#replace anything with exactly 4 backslashes(\\\\) type stuff to 'convertedwindowspath'
	r'\\\\.*?(?= )': 'convertedwindowspath',
	
	#replace ".main.tva.gov" + anything after it to 'convertedserveraddress' these are server names
	r'\b\S+\.main\.tva\.gov\S*\b': 'convertedserveraddressurl',

	#replace the specific server names to 'convertedservername' (ex.  'chapctxxa720 - heartbeat failure - chapctxxa720.main.tva.gov')
	r'(\b\S*)\ - heartbeat failure\b': 'convertedservername  - heartbeat failure',

	#replace all strings starting with 'https:' or 'http:' with 'convertedtvaurl'
	r'https?://\S+': 'convertedtvaurl',

	#replace all strings with 
	r'\S+\.tva\.go\S*': 'covertedtvaurl',

	#replace all tva email addresses (@tva.gov) with 'tvaemailaddress'
	r'\S*@tva\.gov\S*': 'convertedtvaemailaddress',

	#replace 'critialswitchalerts' with spaces in between each word
	r'critialswitchalerts': 'critical swtich alerts',
	
	#r"[A-Za-z0-9\-]{3,50}": ''

}
#preprocessed_hd[col] = preprocessed_hd[col].str.replace(pattern, replacement, regex=True)
